### Installing necessary Libraries 

In [36]:
!wget  https://www.dropbox.com/s/9rw2nnj68zrusn1/amazon_reviews_no_duplicates.csv

--2022-04-15 19:04:37--  https://www.dropbox.com/s/9rw2nnj68zrusn1/amazon_reviews_no_duplicates.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/9rw2nnj68zrusn1/amazon_reviews_no_duplicates.csv [following]
--2022-04-15 19:04:37--  https://www.dropbox.com/s/raw/9rw2nnj68zrusn1/amazon_reviews_no_duplicates.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7a6cdfb55a42bf1226f853cf4f.dl.dropboxusercontent.com/cd/0/inline/Bjd0KoAcr4SNoXwYPNV2tpn-U5_XGuRx80VCQhEa10ZAvFCyJhswkX_06FbGnfg586KkWHU3Ii0uaEFfBNNmRPi2RlTFrcwh4T4-MC3pgJtZNSy0Sf75ecbV-MeqbDZak1EQ33EHmy0rUbXdkIEGSfRaDGpP22uc_sSDQ-0irIV4mA/file# [following]
--2022-04-15 19:04:37--  https://uc7a6cdfb55a42bf1226f853cf4f.dl.dropboxusercontent.com/cd/0/inline/Bjd0KoAc

In [37]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import load_from_disk
import sagemaker
from sagemaker.huggingface import HuggingFace
import random
import logging
import sys
import argparse
import os
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer

In [63]:
dataset = load_dataset('csv', data_files={'train': 'amazon_reviews_no_duplicates.csv'}, encoding = "ISO-8859-1")
remove_columns = ['Unnamed: 0', 'Product Name']
dataset = dataset.map(remove_columns=remove_columns)
dataset = dataset.rename_column("Rating", "stars")
dataset = dataset.rename_column("Reviews", "review_body")

Using custom data configuration default-4be82f884139563e
Reusing dataset csv (/home/ec2-user/.cache/huggingface/datasets/csv/default-4be82f884139563e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/csv/default-4be82f884139563e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-d45b4b84339f0c5a.arrow


In [42]:
dataset

DatasetDict({
    train: Dataset({
        features: ['stars', 'review_body'],
        num_rows: 162492
    })
})

In [43]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

loading configuration file https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment/resolve/main/config.json from cache at /home/ec2-user/.cache/huggingface/transformers/d9226eeac7b8b96d83ebc327cdd670490866d8c999505c1f83b6ef206ccb1604.a34960b447312b0727cb670d710444fcb41a6156eddcba062a19b3fc05d95251
Model config BertConfig {
  "_name_or_path": "nlptown/bert-base-multilingual-uncased-sentiment",
  "_num_labels": 5,
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "finetuning_task": "sentiment-analysis",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "1 star",
    "1": "2 stars",
    "2": "3 stars",
    "3": "4 stars",
    "4": "5 stars"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "1 star": 0,
    "2 stars": 1,
    "3 stars": 2,
    "4 stars": 3,
    "5 stars": 4
  },
  "l

In [64]:
def preprocess_data(s):
    if s['review_body'] == None or s['stars'] == None :
        return False
    else:
        return True
def tokenize_data(s):
    #print(s)
    return tokenizer(s['review_body'], padding='max_length',truncation=True, max_length=512)

def transform_labels(s): 
    
    if int(s['stars']) == 1:
        l = "1 star"
    else:
        l = str(s['stars'])+" stars"
    return {'label': l}
    

def int_labels(s):
    
    return {'label':int(s['stars'])-1}

In [65]:
dataset = dataset.filter(preprocess_data)
dataset = dataset.map(tokenize_data)
dataset = dataset.map(int_labels)

Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/csv/default-4be82f884139563e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-fd0c9752e154ad20.arrow


  0%|          | 0/162491 [00:00<?, ?ex/s]

  0%|          | 0/162491 [00:00<?, ?ex/s]

In [66]:
dataset = dataset.map(remove_columns=['stars','review_body'])

  0%|          | 0/162491 [00:00<?, ?ex/s]

In [67]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 162491
    })
})

In [68]:
training_args = TrainingArguments("test_trainer", num_train_epochs=3,per_device_train_batch_size=4)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [69]:
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment", num_labels=5)

loading configuration file https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment/resolve/main/config.json from cache at /home/ec2-user/.cache/huggingface/transformers/d9226eeac7b8b96d83ebc327cdd670490866d8c999505c1f83b6ef206ccb1604.a34960b447312b0727cb670d710444fcb41a6156eddcba062a19b3fc05d95251
Model config BertConfig {
  "_name_or_path": "nlptown/bert-base-multilingual-uncased-sentiment",
  "_num_labels": 5,
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "finetuning_task": "sentiment-analysis",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "1 star",
    "1": "2 stars",
    "2": "3 stars",
    "3": "4 stars",
    "4": "5 stars"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "1 star": 0,
    "2 stars": 1,
    "3 stars": 2,
    "4 stars": 3,
    "5 stars": 4
  },
  "l

In [75]:
train_dataset = dataset['train'].shuffle(seed=42).select(range(10000))
eval_dataset = dataset['train'].shuffle(seed=42).select(range(1000))

Loading cached shuffled indices for dataset at /home/ec2-user/.cache/huggingface/datasets/csv/default-4be82f884139563e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-bb5b1f4b55251bb5.arrow
Loading cached shuffled indices for dataset at /home/ec2-user/.cache/huggingface/datasets/csv/default-4be82f884139563e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-bb5b1f4b55251bb5.arrow


In [77]:
trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

In [52]:
torch.autograd.set_detect_anomaly(True)

In [79]:
trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10000
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 7500


Step,Training Loss
500,1.080000
1000,1.010400
1500,1.014300
2000,0.939100
2500,0.973600
3000,0.900800
3500,0.860200
4000,0.816100
4500,0.859500
5000,0.853000


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2500
Configuration saved in test_trainer/checkpoint-2500/config.json
Model weights saved in test_trainer/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-30

TrainOutput(global_step=7500, training_loss=0.8532799682617187, metrics={'train_runtime': 4508.9451, 'train_samples_per_second': 6.653, 'train_steps_per_second': 1.663, 'total_flos': 7893544273920000.0, 'train_loss': 0.8532799682617187, 'epoch': 3.0})